# Lesson 5.4: Custom Memory and Chatbot Practice

---

In previous lessons, we explored LangChain's built-in **Memory** types, from basic ones like `ConversationBufferMemory` to more advanced ones like `ConversationSummaryMemory`. However, in some cases, you might need a more specialized way to store or process conversation history. This lesson will guide you on how to create **Custom Memory** and apply all your knowledge about Memory to build a practical chatbot.

## 1. Creating Custom Memory Types

### 1.1. When is Custom Memory Needed?

You should consider creating Custom Memory when:

* **Integration with External Databases:** You want to store conversation history in a specific database (SQL, NoSQL, Redis, etc.) instead of just in application memory.
* **Special Storage/Retrieval Logic:** You need more complex logic to decide which messages should be stored, when they should be deleted, or how to retrieve them.
* **Data Pre/Post-processing:** You want to perform data processing steps before saving to or after retrieving from memory (e.g., encryption, compression, filtering).
* **Integration with Other State Management Systems:** You already have a user state management system and want LangChain Memory to utilize it.



### 1.2. How to Create Custom Memory

To create Custom Memory, you will typically inherit from LangChain's `BaseMemory` class and implement the necessary methods. The most important methods include:

* `load_memory_variables(inputs: Dict[str, Any]) -> Dict[str, Any]`: This method is called to load conversation history from memory and return it as a dictionary.
* `save_context(inputs: Dict[str, Any], outputs: Dict[str, Any]) -> None`: This method is called to store user input and AI output into memory after each conversation turn.
* `clear() -> None`: This method is called to clear the entire history in memory.

You also need to define the input (`input_keys`) and output (`output_keys`) variables that Memory will manage.

**Example: Simple Custom Memory (storing in a Python list)**

Here's an example of Custom Memory that stores conversation history in a simple Python list.

In [ ]:
from typing import Any, Dict, List, Optional
from langchain_core.memory import BaseMemory
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage

class SimpleListMemory(BaseMemory):
    """
    Custom memory that stores conversation history as a list of messages.
    """
    conversation_history: List[BaseMessage] = []
    input_key: str = "input"
    output_key: str = "output"

    @property
    def memory_variables(self) -> List[str]:
        """Variables that the memory will return."""
        return ["history"] # Name of the variable the prompt will use to get history

    def load_memory_variables(self, inputs: Dict[str, Any]) -> Dict[str, Any]:
        """Load conversation history from memory."""
        # Convert the list of messages into a formatted string for the prompt
        formatted_history = ""
        for msg in self.conversation_history:
            if isinstance(msg, HumanMessage):
                formatted_history += f"Human: {msg.content}\n"
            elif isinstance(msg, AIMessage):
                formatted_history += f"AI: {msg.content}\n"
        return {"history": formatted_history.strip()}

    def save_context(self, inputs: Dict[str, Any], outputs: Dict[str, Any]) -> None:
        """Store input and output into memory."""
        user_message = inputs.get(self.input_key)
        ai_message = outputs.get(self.output_key)

        if user_message:
            self.conversation_history.append(HumanMessage(content=user_message))
        if ai_message:
            self.conversation_history.append(AIMessage(content=ai_message))

    def clear(self) -> None:
        """Clear the entire conversation history."""
        self.conversation_history = []

# Test Custom Memory
print("--- Kiểm tra Custom Memory ---") # Test Custom Memory
custom_memory = SimpleListMemory()
custom_memory.save_context({"input": "Chào bạn"}, {"output": "Tôi khỏe, bạn thế nào?"}) # Hello, how are you? / I'm fine, how about you?
custom_memory.save_context({"input": "Bạn tên gì?"}, {"output": "Tôi là một mô hình ngôn ngữ."}) # What's your name? / I am a language model.
print(custom_memory.load_memory_variables({}))
custom_memory.clear()
print(custom_memory.load_memory_variables({}))
print("-" * 30)

### 1.3. How to Access and Manipulate Data in Memory

Once you've integrated Memory into a Chain or Agent, accessing and manipulating data in Memory is done through the Memory object's methods:

* `memory.load_memory_variables({})`: To get the current content of the memory.
* `memory.save_context(inputs, outputs)`: To add a new conversation turn to memory.
* `memory.clear()`: To clear the memory.

In the following practical examples, you'll see how Chains and Agents automatically call these methods.


---

## 2. Practical Example: Building a Chatbot Capable of Maintaining Context

We will build a simple chatbot and test it with different Memory types to see the differences in how they maintain context.

**Preparation:**
* Ensure you have `langchain-openai` installed.
* Set the `OPENAI_API_KEY` environment variable.

In [ ]:
# Install libraries if not already installed
# pip install langchain-openai openai

import os
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import (
    ConversationBufferMemory,
    ConversationBufferWindowMemory,
    ConversationSummaryMemory,
    ConversationSummaryBufferMemory,
)
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# Set environment variable for OpenAI API key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# Initialize LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0.7)

# Define a function to run the chatbot with a specific Memory type
def run_chatbot_with_memory(memory_instance, memory_name: str, num_turns: int = 5):
    """
    Runs a chatbot with the specified Memory type.
    """
    print(f"\n--- Bắt đầu Chatbot với {memory_name} ---") # Starting Chatbot with {memory_name}

    # Basic prompt for the chatbot
    prompt_template = ChatPromptTemplate.from_messages([
        ("system", "Bạn là một trợ lý hữu ích. Hãy trả lời các câu hỏi của người dùng. Duy trì ngữ cảnh cuộc trò chuyện."), # You are a helpful assistant. Answer user questions. Maintain conversation context.
        MessagesPlaceholder(variable_name="history"), # Placeholder for conversation history
        ("human", "{input}"),
    ])

    conversation_chain = ConversationChain(
        llm=llm,
        memory=memory_instance,
        prompt=prompt_template,
        verbose=False # Set False to avoid cluttered output, but you can set True for debugging
    )

    # Sample questions to test context
    questions = [
        "Chào bạn, tôi là An. Tôi đang tìm hiểu về LangChain.", # Hello, I'm An. I'm learning about LangChain.
        "Bạn có thể giải thích về RAG không?", # Can you explain RAG?
        "Vậy RAG giúp ích gì?", # So what does RAG help with?
        "Tên tôi là gì?", # What's my name?
        "Bạn có thể nhắc lại các thành phần chính của LangChain không?", # Can you reiterate the main components of LangChain?
        "Tôi có thể sử dụng RAG với loại cơ sở dữ liệu nào?", # What kind of databases can I use RAG with?
    ]

    for i in range(min(num_turns, len(questions))):
        user_input = questions[i]
        print(f"Người dùng ({i+1}): {user_input}") # User ({i+1}):
        response = conversation_chain.invoke({"input": user_input})
        print(f"AI ({i+1}): {response['response']}") # AI ({i+1}):
        print("-" * 20)

    print(f"--- Kết thúc Chatbot với {memory_name} ---") # Ending Chatbot with {memory_name}
    print(f"Nội dung cuối cùng của {memory_name}:") # Final content of {memory_name}:
    print(memory_instance.buffer if hasattr(memory_instance, 'buffer') else memory_instance.load_memory_variables({}))
    print("=" * 50)

# 2.1. Use ConversationBufferMemory
buffer_memory = ConversationBufferMemory()
run_chatbot_with_memory(buffer_memory, "ConversationBufferMemory")

# 2.2. Use ConversationBufferWindowMemory (k=2)
window_memory = ConversationBufferWindowMemory(k=2)
run_chatbot_with_memory(window_memory, "ConversationBufferWindowMemory (k=2)")

# 2.3. Use ConversationSummaryMemory
# Requires an LLM for summarization
summary_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
summary_memory = ConversationSummaryMemory(llm=summary_llm)
run_chatbot_with_memory(summary_memory, "ConversationSummaryMemory")

# 2.4. Use ConversationSummaryBufferMemory (max_token_limit=100)
# Requires an LLM for summarization
summary_buffer_llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
summary_buffer_memory = ConversationSummaryBufferMemory(
    llm=summary_buffer_llm,
    max_token_limit=100
)
run_chatbot_with_memory(summary_buffer_memory, "ConversationSummaryBufferMemory (max_token_limit=100)")

# 2.5. Use Custom Memory (SimpleListMemory)
custom_memory_instance = SimpleListMemory()
run_chatbot_with_memory(custom_memory_instance, "SimpleListMemory")

## 3. Comparing the Effectiveness of Different Memory Types in Various Scenarios

After running the code above, you will observe clear differences in how each Memory type handles context:

* **`ConversationBufferMemory`**:
    * **Effectiveness:** Maintains perfect context for short conversations.
    * **Limitations:** As the conversation lengthens, the `history` in the prompt will grow, leading to increased token costs and a risk of exceeding the LLM's token limits. The LLM can also become "overwhelmed" with information.
    * **When to Use:** Short interactive applications where you need the full history and are not concerned about token limits/costs.

* **`ConversationBufferWindowMemory` (k=2)**:
    * **Effectiveness:** Good control over prompt size and token costs by only retaining the last `k` turns.
    * **Limitations:** Loses long-term context. You will notice the LLM forgets the name "An" in turn 4, as that information has been pushed out of the window.
    * **When to Use:** Applications requiring short-term context, or when you know that information older than a certain threshold is no longer relevant.

* **`ConversationSummaryMemory`**:
    * **Effectiveness:** Retains long-term context as a summary, significantly reducing the number of tokens passed to the LLM in subsequent turns.
    * **Limitations:** Incurs additional cost and latency due to LLM calls for summarization. The quality of the summary depends on the LLM. Some specific details not included in the summary might be lost.
    * **When to Use:** Very long conversations where you need to maintain general context without overflowing the token buffer.

* **`ConversationSummaryBufferMemory` (max_token_limit=100)**:
    * **Effectiveness:** Provides a good balance between retaining recent details and summarizing older context. This is often the optimal choice for many practical chatbots.
    * **Limitations:** Still has cost and latency associated with summarization, and quality depends on the LLM.
    * **When to Use:** Most chatbots that need to maintain long-term context but still want to control costs and performance.

* **`SimpleListMemory` (Custom Memory)**:
    * **Effectiveness:** Illustrates how you can fully customize the storage logic. In this example, it behaves similarly to `ConversationBufferMemory`, but you could modify it to store in a database or apply more complex logic.
    * **Limitations:** Depends on the logic you define. If not careful, you might reintroduce token/context issues similar to basic memory types.
    * **When to Use:** When no built-in Memory type meets your specific data storage/processing requirements.

Choosing the appropriate Memory type is a crucial decision in your chatbot's design. It depends on the average conversation length, context accuracy requirements, LLM token limits, and your budget.


---

## Lesson Summary

In this lesson, you learned how to create **Custom Memory** in LangChain by inheriting from `BaseMemory` and implementing the necessary methods, opening up possibilities for integration with custom storage systems and logic. You also practiced **building a chatbot** and testing it with different Memory types: `ConversationBufferMemory`, `ConversationBufferWindowMemory`, `ConversationSummaryMemory`, and `ConversationSummaryBufferMemory`. Finally, we **compared the effectiveness** of each Memory type in various scenarios, helping you understand their pros and cons and make an informed choice for your application. Mastering Memory types and the ability to customize them is key to building intelligent and effective conversational applications.